# Price Discrimination Scrape - Confirmatory Analysis
This is the code for doing the price discrimination scrape. It is comprised of two steps:
- Getting Amazon URLS
- Getting Data from URLS

> Note that due to ethics reasons we will not be storing this data and the data is removed from the MongoDB records.

In [ ]:
# Imports
import pandas as pd
import requests
from pymongo import MongoClient
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import concurrent.futures

### Load in the data

In [ ]:
df = pd.read_csv("archive/asr.csv")

In [21]:
skus = list(df.SKU.unique())
skus[0]

'SET389-KR-NP-S'

### Retrieving the SKU Amazon Page Urls
Retrieve the SKUs of the Amazon pages per SKU for the next stage of scraping
- Loop over all skus
- Get the page with the driver
- Once retrieved, get the soup source
- Use getData for all of the results (.g)
- Store data in mongo collection


> Note that Captchas still need to be solved

In [ ]:
skus = [x for x in client.ba.skus.find({'retrieved': 0})]
driver = webdriver.Chrome()

for i in skus:
    sku = i['sku']
    driver.get(f'https://www.google.com/search?hl=en&q={sku}')
    soup = BeautifulSoup(driver.page_source, 'lxml')
    link_list = soup.select(".g")
    
    if "It looks like there aren't" in soup.text:
        client.ba.skus.update_one({'sku': sku}, {"$set": {"retrieved": 2}})
        continue
        
    if 'Our systems have detected unusual traffic from your computer' in soup.text:
        # driver.close()
        # driver = webdriver.Chrome()
        input("Do the captcha and click enter")
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    for elem in link_list:
        d = getData(elem, sku)
        client.ba.links.insert_one(d)
    client.ba.skus.update_one({'sku': sku}, {"$set": {"retrieved": 1}})
    

### Retrieving the price from Amazon
Using a proprietary scraping api, pricing is retrieved and stored in a new MongoDB collection

In [1]:
def request_url(url, params):
    tries = 0
    page = ''
    while page == '' and tries < 10:
        try:
            page = requests.get(url, params=params)
            return page
        except:
            tries += 1
            time.sleep(3)
            continue
    return None


def proprietarySoftware():
    return ""


def getPage(url, tries=0):
    if tries > 10:
        return None, None

    res = proprietarySoftware()
    
    soup = BeautifulSoup(res.content, 'html.parser')
    if soup is None:
        return None, None
    if len(soup.select("#desktop_unifiedPrice")) < 1:
        soup, tries = getPage(url, tries=tries+1)

    return soup, tries

Retrieve the different data parts (due to different page structures for different countries, different prices are retrieved

In [2]:
def getData(soup, link):
    data = {'link': link, 'data_retrieved': 1}
    
    try:
        data['title'] = soup.select("#productTitle")[0].text.strip()
    except:
        pass
    try:
        data['n_ratings'] = soup.select("#acrCustomerReviewText")[0].text
    except:
        pass
    try:
        data['price'] = soup.select('.a-price-symbol')[0].text + soup.select(".a-price-whole")[0].text + soup.select(".a-price-fraction")[0].text
    except:
        pass
    try:
        data['price2'] = soup.select('#desktop_unifiedPrice')[0].text
    except:
        pass
    try:
        data['rating'] = soup.select("#acrPopover")[0].text
    except:
        pass
    try:
        data['other'] = soup.select("#productFactsDesktopExpander")[0].text
    except:
        pass
    
    return data
    

Loop over the different skus and then the retrieved URLS to get the data and input it into the MongoDB

In [ ]:
def getSkuData(skus):
    client = MongoClient()
    for sku in skus:
        links = [x for x in client.ba.links.find({'sku': sku})]
        link_found = False
        for i in links:
            if link_found:
                break
            link = i['link']
            if '/product-reviews/' in link:
                continue
            if 'www.amazon' in link:
                link_found = True
                # Do the function
                soup, _ = getPage(link)
                if soup is None:
                    continue
                data = getData(soup, link)
                data['sku'] = sku
                client.ba.skus.update_one({'sku': sku}, {"$set": data})
                client.ba.sku_data.insert_one(data)
        if not link_found:
            client.ba.skus.update_one({'sku': sku}, {"$set": {'data_retrieved': 1}})
    client.close()
    return True  

Run the function concurrently with 50 workers to allow for a quicker retrieval (~1 hour)

In [ ]:
skus_found = client.ba.sku.find({"data_retrieved": {"$nin": [1]}}).distinct("sku")
skus = client.ba.links.find({'sku': {"$nin": skus_found}}).distinct("sku")

split_links = np.array_split(skus, 50)

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    lnks = [executor.submit(getSkuData, docs) for docs in split_links]

### Data Comparison
A simple median check is done to see if the data is similar in both datasets.

#### Data Loading and Cleaning
- Load the data from the Mongo Database
- Clear currency and translate to one common currency (of that day)

In [ ]:
sku_data = []

for sku in skus:
    d = client.ba.sku_data.find_one({'sku': sku})
    sku_data.append(d)

In [161]:
df = pd.DataFrame.from_dict(sku_data)
df.head()

,_id,link,data_retrieved,title,price,price2,sku,n_ratings,rating,other
0,65417a27c2bd975958033dba,https://www.amazon.com/KASK-Safety-Helmet-Zeni...,1,"KASK Safety Helmet Zenith XL HI VIZ - an, 201-...",$159.99,\n\n\n,AN201-RED-XL,NaN,NaN,NaN
1,6541ad34c2bd97595803588b,https://www.amazon.com/KASK-Safety-Helmet-Zeni...,1,"KASK Safety Helmet Zenith XL HI VIZ - an, 201-...",$159.99,\n\n\n,AN201-RED-XXL,NaN,NaN,NaN
2,65417ab9c2bd975958033e9b,https://www.amazon.com/Doctor-Stuff-Veterinari...,1,Doctor Stuff - Veterinarian File Folder Chart ...,$12.39,\n\n\n,AN204-PURPLE-S,4 ratings,\n 5.0 5.0 out of 5 stars,NaN
3,65415f85c2bd97595803372c,https://www.amazon.com/ZZYINH-Organizer-Neckla...,1,ZZYINH AN207 Round Velvet Jewelry Organizer Ri...,NaN,\n\n\n,AN207-PINK-L,NaN,NaN,NaN
4,65417b8bc2bd975958033f8b,https://www.amazon.com/ZZYINH-Organizer-Neckla...,1,ZZYINH AN207 Round Velvet Jewelry Organizer Ri...,NaN,\n\n\n,AN207-PINK-S,NaN,NaN,NaN


In [ ]:
curr_lookup = {
    '$': 'USD',
    '₹': 'RUP', 
    'SAR': 'SAR', 
    '£': 'GBP', 
    '€': 'EUR', 
    'AED': 'AED', 
    'S$': 'USD', 
    '$\u200e': 'USD', 
    'EGP': 'EGP',
    'US$': 'USD', 
    'kr': 'SKR', 
    '¥': 'YCN'
}

def parsePrice(price, curr=False):
    if type(price) is not str:
        return np.nan
    
    parsed_price = ''.join(re.findall(r'[0-9\.\,]', price))
    p_t = price
    
    for elem in parsed_price:
        p_t = p_t.replace(elem, '')
    currency = p_t
    
    if curr:
        try:
            return curr_lookup[currency]
        except:
            return np.nan
    
    if ',' in parsed_price:
        if '.' in parsed_price:
            # Here it is used as a 1.000 separator
            parsed_price = parsed_price.replace(",", "")
        else:
            parsed_price = parsed_price.replace(",", ".")
            
    if len(parsed_price) > 4:
        parsed_price = parsed_price[:3]
    
    return float(parsed_price)


df['price_parsed'] = df.apply(lambda x: parsePrice(x['price']), axis=1)
df['currency'] = df.apply(lambda x: parsePrice(x['price'], True), axis=1)

In [ ]:
curr_lookup = {
    '$': 1.06,
    '₹': 88.4, 
    'SAR': 3.98, 
    '£': 0.87, 
    '€': 1, 
    'AED': 3.9, 
    'S$': 1.06, 
    '$\u200e': 1.06, 
    'EGP': 32.86,
    'US$': 1.06, 
    'kr': 11.81, 
    '¥': 7.77
}

def updatePriceEur(price, currency):
    try:
        return price * currency
    except:
        return price
    
    
df['price_euro'] = df.apply(lambda x: updatePriceEur(x['price_parsed'], x['currency']), axis=1)

Now load in the data and add the found price to the dataset to do a comparison

In [ ]:
data = pd.read_csv('asr.csv')

def addPrice(sku):
    try:
        return sku_lookup[sku]
    except:
        return np.nan
    
data['price'] = data.apply(lambda x: addPrice(x['SKU']), axis=1)

In [219]:
data['price'].describe()

count    67102.000000
mean       170.118558
std        227.067492
min          1.000000
25%         26.000000
50%         50.000000
75%        299.000000
max        999.000000
Name: price, dtype: float64

In [220]:
data[data['Status'].isin(['Shipped - Returned to Seller', 'Shipped - Rejected by Buyer'])]['price'].describe()

count    939.000000
mean     185.005911
std      233.392637
min        1.300000
25%       27.000000
50%       55.000000
75%      339.000000
max      999.000000
Name: price, dtype: float64

In [4]:
data['Amount'].describe()

count    121180.000000
mean        648.561465
std         281.211687
min           0.000000
25%         449.000000
50%         605.000000
75%         788.000000
max        5584.000000
Name: Amount, dtype: float64

### Conclusion
One can see that the mean is around the same for both prices and thus our confirmatory experiment does indeed confirm that the prices in the dataset are correct. Additionally one can see that products of a lower price are send back more often (mean is significantly lower)